Following the tutorial for [how to design a part in build123d](https://build123d.readthedocs.io/en/stable/tutorial_design.html)

In [42]:
from build123d import (
    Axis,
    BuildLine,
    BuildPart,
    BuildSketch,
    FilletPolyline,
    Hole,
    Locations,
    MM,
    Mode,
    Plane,
    Side,
    SlotOverall,
    extrude,
    fillet,
    make_face,
    mirror,
    offset,
)
from ocp_vscode import show

# Parameters for the bracket
thickness = 3 * MM
width = 25 * MM
length = 50 * MM
height = 25 * MM
hole_diameter = 5 * MM
slot_length = 20 * MM
bend_radius = 5 * MM
fillet_radius = 2 * MM


1. Create a line using `FilletPolyline` with two parts and a curved corner
1. Turn the line into a 2D shape using offset, which gives the line "thickness"
1. Mirror the 2D corner shape to create a "U" shape

In [43]:
with BuildSketch() as sketch:
    with BuildLine() as profile:
        FilletPolyline(
            (0, 0), (length / 2, 0), (length / 2, height), radius=bend_radius
        )
        offset(amount=thickness, side=Side.LEFT)
    make_face()
    mirror(about=Plane.YZ)

show(sketch)

-c


1. Take the 2D object from the previous step and use `extrude` to extend it out into a 3D object.
1. Use mirror again, but this time to copy the object, so it's symmetrical (it already was, but worth checking)

In [44]:
with BuildPart() as bracket:
    with BuildSketch() as sketch:
        with BuildLine() as profile:
            FilletPolyline(
                (0, 0), (length / 2, 0), (length / 2, height), radius=bend_radius
            )
            offset(amount=thickness, side=Side.LEFT)
        make_face()
        mirror(about=Plane.YZ)
    extrude(amount=width / 2)
    mirror(about=Plane.XY)

show(bracket)

c


1. Select sharp corners at the tips of the U-shape of the bracket
1. Fillet the corners

In [45]:

with BuildPart() as bracket:
    with BuildSketch() as sketch:
        with BuildLine() as profile:
            FilletPolyline(
                (0, 0), (length / 2, 0), (length / 2, height), radius=bend_radius
            )
            offset(amount=thickness, side=Side.LEFT)
        make_face()
        mirror(about=Plane.YZ)
    extrude(amount=width / 2)
    mirror(about=Plane.XY)
    corners = bracket.edges().filter_by(Axis.X).group_by(Axis.Y)[-1]
    fillet(corners, fillet_radius)

show(bracket)

c


Still working inside the bracket part:
1. Use Locations to select the ends of the bracket
1. Cut holes in the centre of the ends of the bracket

In [46]:
with BuildPart() as bracket:
    with BuildSketch() as sketch:
        with BuildLine() as profile:
            FilletPolyline(
                (0, 0), (length / 2, 0), (length / 2, height), radius=bend_radius
            )
            offset(amount=thickness, side=Side.LEFT)
        make_face()
        mirror(about=Plane.YZ)
    extrude(amount=width / 2)
    mirror(about=Plane.XY)
    corners = bracket.edges().filter_by(Axis.X).group_by(Axis.Y)[-1]
    fillet(corners, fillet_radius)

    with Locations(bracket.faces().sort_by(Axis.X)[-1]):
        Hole(hole_diameter / 2)

show(bracket)

c


Finally, still working on the bracket Part, cut out the slot
1. Create a 2D Sketch using SlotOverall (I replaced the hard-coded dimension of the slot with a variable)
1. Use `extrude` with the mode set to substract to cut the slot from the chosen wall

In [47]:
with BuildPart() as bracket:
    with BuildSketch() as sketch:
        with BuildLine() as profile:
            FilletPolyline(
                (0, 0), (length / 2, 0), (length / 2, height), radius=bend_radius
            )
            offset(amount=thickness, side=Side.LEFT)
        make_face()
        mirror(about=Plane.YZ)
    extrude(amount=width / 2)
    mirror(about=Plane.XY)
    corners = bracket.edges().filter_by(Axis.X).group_by(Axis.Y)[-1]
    fillet(corners, fillet_radius)

    with Locations(bracket.faces().sort_by(Axis.X)[-1]):
        Hole(hole_diameter / 2)

    with BuildSketch(bracket.faces().sort_by(Axis.Y)[0]):
        SlotOverall(slot_length * MM, hole_diameter)
    extrude(amount=-thickness, mode=Mode.SUBTRACT)
show(bracket)

+
